In [5]:
# fundemental modules
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import time
import pprint
from tqdm.notebook import trange

from ddqn import DDQNAgent # type: ignore
from utils import load_config
from obs import observation_shape

aggregate_stats_every=100
best_score = -1000.0
score_history = []
episode_lens = []
avg_history = []
std_history = []
avg_history_100 = []



In [6]:
data = load_config()
env = gym.make('racetrack-v0', render_mode = 'rgb_array')
env.configure(data) # type: ignore
# pprint.pprint(env.config) # type: ignore
(obs, info), done = env.reset(), False

#observation config
proc = observation_shape(obs,info,2)
proc.reset()
input = proc.get_input()
print(input.shape)


(26,)


/home/o/Documents/thesis/.venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.configure to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.configure` for environment variables or `env.get_wrapper_attr('configure')` that will search the reminding wrappers.
  logger.warn(


In [7]:
agent = DDQNAgent(alpha=0.001, gamma=0.9, epsilon=1.0, obs_shape=input.shape,
                  batch_size=64, epsilon_dec=0.9995, epsilon_end=0.05, mem_size=100000,
                  min_mem_size=100, replace_target=1000, learning_rate=0.0003)
print("Agent is initialized.")


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 128)               3456      
                                                                 
 dense_7 (Dense)             (None, 128)               16512     
                                                                 
 dense_8 (Dense)             (None, 11)                1419      
                                                                 
Total params: 21387 (83.54 KB)
Trainable params: 21387 (83.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
summary
Agent is initialized.


In [8]:
# main loop
for episode in trange(500, desc='Test episodes'):

        proc.reset()
        (observation, info), done = env.reset(), False
        truncated = False
        proc.update_input(observation, info)
        observation = proc.get_input()

        episode_reward = 0
        episode_len = 0

        while (not done) and (not truncated):
            
            action, action_index = agent.get_action(observation, deterministic=False)
            new_observation, reward, done, truncated, new_info = env.step(action=[action])
            
            proc.update_input(new_observation, info)
            new_observation = proc.get_input()

            episode_reward += reward # type: ignore
            episode_len +=1

            if new_info["rewards"]["on_road_reward"] == False or episode_len >= 200:
                done = True
            
            agent.remember(state=observation, action=action_index, done=done,
                            reward=reward, new_state=new_observation)
            agent.train()

            observation = new_observation

        episode_lens.append(episode_len)

        score_history.append(episode_reward)
        avg_score = np.mean(score_history)
        avg_history.append(avg_score)
        std_score = np.std(score_history)
        std_history.append(std_score)

        avg_score_100 = np.mean(score_history[-100:])
        avg_history_100.append(avg_score_100)

        if avg_score_100 > best_score:
            best_score = avg_score
            agent.save_model(episode)

        agent.tensorboard.update_stats(episode_rew = episode_reward,
                                       average_rew =avg_score,
                                       average_100_reward = avg_score_100,
                                       std_rew=std_score,
                                       epsilon=agent.epsilon,
                                       episode_len = episode_len)
        
        # #* save things in checkpoints
        # if not episode % aggregate_stats_every or episode == 1:
        #     average_reward = sum(score_history[-aggregate_stats_every:])/len(score_history[-aggregate_stats_every:])
        #     min_reward = min(score_history[-aggregate_stats_every:])
        #     max_reward = max(score_history[-aggregate_stats_every:])

        #     agent.tensorboard_steps.update_stats(reward_avg_steps=average_reward, reward_min_steps=min_reward, reward_max_steps=max_reward, epsilon_steps=agent.epsilon)

        print('episode ', episode, 'score %.1f' % episode_reward, 'ep len', episode_len,
              'avg score %.1f' % avg_score, 'avg_score_100 %.1f' %avg_score_100,'std score %.1f' % std_score)

Test episodes:   0%|          | 0/500 [00:00<?, ?it/s]

-----saving models------


2023-12-02 21:50:13.966874: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


episode  0 score 0.6 ep len 7 avg score 0.6 avg_score_100 0.6 std score 0.0
episode  1 score -0.5 ep len 16 avg score 0.1 avg_score_100 0.1 std score 0.5
episode  2 score -0.6 ep len 12 avg score -0.2 avg_score_100 -0.2 std score 0.5
episode  3 score -0.2 ep len 8 avg score -0.2 avg_score_100 -0.2 std score 0.5
episode  4 score 0.4 ep len 18 avg score -0.1 avg_score_100 -0.1 std score 0.5
episode  5 score -0.2 ep len 4 avg score -0.1 avg_score_100 -0.1 std score 0.4
episode  6 score 1.1 ep len 6 avg score 0.1 avg_score_100 0.1 std score 0.6
episode  7 score 0.1 ep len 14 avg score 0.1 avg_score_100 0.1 std score 0.5
episode  8 score -0.5 ep len 3 avg score 0.0 avg_score_100 0.0 std score 0.5
episode  9 score 0.3 ep len 10 avg score 0.0 avg_score_100 0.0 std score 0.5
episode  10 score 1.1 ep len 5 avg score 0.1 avg_score_100 0.1 std score 0.6
episode  11 score -0.3 ep len 2 avg score 0.1 avg_score_100 0.1 std score 0.6
episode  12 score 0.1 ep len 12 avg score 0.1 avg_score_100 0.1 std

2023-12-02 21:50:17.330816: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f4a5c02c650 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-02 21:50:17.330829: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-12-02 21:50:17.335994: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-02 21:50:18.574831: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2023-12-02 21:50:18.632092: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


episode  31 score 0.5 ep len 39 avg score 0.2 avg_score_100 0.2 std score 1.0
episode  32 score 0.8 ep len 21 avg score 0.2 avg_score_100 0.2 std score 1.0
episode  33 score -0.5 ep len 9 avg score 0.2 avg_score_100 0.2 std score 1.0
episode  34 score 3.0 ep len 12 avg score 0.3 avg_score_100 0.3 std score 1.1
episode  35 score -1.6 ep len 18 avg score 0.3 avg_score_100 0.3 std score 1.1
episode  36 score 1.7 ep len 13 avg score 0.3 avg_score_100 0.3 std score 1.1
episode  37 score 0.1 ep len 17 avg score 0.3 avg_score_100 0.3 std score 1.1
episode  38 score 1.7 ep len 17 avg score 0.3 avg_score_100 0.3 std score 1.1
episode  39 score -1.2 ep len 11 avg score 0.3 avg_score_100 0.3 std score 1.1
episode  40 score -0.3 ep len 3 avg score 0.3 avg_score_100 0.3 std score 1.1
episode  41 score -0.3 ep len 2 avg score 0.3 avg_score_100 0.3 std score 1.1
episode  42 score -0.3 ep len 3 avg score 0.2 avg_score_100 0.2 std score 1.1
episode  43 score -2.0 ep len 17 avg score 0.2 avg_score_100 0

KeyboardInterrupt: 

In [9]:
n_steps = sum(episode_lens)
print(n_steps)


24663


In [12]:
# env = record_videos(env)
from gymnasium.wrappers import RecordVideo

data = load_config()
env = gym.make('racetrack-v0', render_mode = 'rgb_array')
env.configure(data) # type: ignore

env = RecordVideo(env, video_folder="videos", episode_trigger=lambda e: True)
env.unwrapped.set_record_video_wrapper(env)

# main loop
for episode in trange(3, desc='Test episodes'):

        proc.reset()
        (observation, info), done = env.reset(), False
        truncated = False
        proc.update_input(observation, info)
        observation = proc.get_input()

        episode_reward = 0
        episode_len = 0

        while (not done) or truncated:
            
            action, action_index = agent.get_action(observation, deterministic=True)
            new_observation, reward, done, truncated, new_info = env.step(action=[action])
            
            proc.update_input(new_observation, info)
            new_observation = proc.get_input()

            episode_reward += reward # type: ignore
            episode_len +=1

            if new_info["rewards"]["on_road_reward"] == False or episode_len >=150:
               done = True
            
            observation = new_observation

        print('episode ', episode, 'score %.1f' % episode_reward, 'ep len', episode_len)   

env.close()     

/home/o/Documents/thesis/.venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.configure to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.configure` for environment variables or `env.get_wrapper_attr('configure')` that will search the reminding wrappers.
  logger.warn(
/home/o/Documents/thesis/.venv/lib/python3.10/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /home/o/Documents/thesis/ddqn/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Test episodes:   0%|          | 0/3 [00:00<?, ?it/s]

episode  0 score 128.1 ep len 150
episode  1 score 123.4 ep len 146


KeyboardInterrupt: 